## Load Data

In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [167]:
credit=pd.read_csv('cc_info.csv')
trans=pd.read_csv('transactions.csv',parse_dates=['date'])

In [168]:
credit.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [169]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
credit_card          984 non-null int64
city                 984 non-null object
state                984 non-null object
zipcode              984 non-null int64
credit_card_limit    984 non-null int64
dtypes: int64(3), object(2)
memory usage: 38.5+ KB


In [170]:
trans.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [171]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 5 columns):
credit_card                  294588 non-null int64
date                         294588 non-null datetime64[ns]
transaction_dollar_amount    294588 non-null float64
Long                         294588 non-null float64
Lat                          294588 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 11.2 MB


## Question 1

### identify those users that in your dataset never went above the monthly credit card limit 􏰋(􏰌calendar 􏰍month)􏰎.

In [172]:
# extract month, weekday, and hour information
trans['month'] = trans['date'].apply(lambda x: x.month)
trans['day'] = trans['date'].apply(lambda x: x.day)
trans['weekday'] = trans['date'].apply(lambda x: x.dayofweek)
trans['hour'] = trans['date'].apply(lambda x: x.hour)

# drop useless features
trans = trans.drop(labels='date', axis=1)

In [173]:
trans.head()

,credit_card,transaction_dollar_amount,Long,Lat,month,day,weekday,hour
0,1003715054175576,43.78,-80.174132,40.267370,9,11,4,0
1,1003715054175576,103.15,-80.194240,40.180114,10,24,5,22
2,1003715054175576,48.55,-80.211033,40.313004,10,26,0,18
3,1003715054175576,136.18,-80.174138,40.290895,10,22,3,19
4,1003715054175576,71.82,-80.238720,40.166719,10,26,0,20


In [174]:
trans_sum=trans[['transaction_dollar_amount','credit_card','month']].groupby(['credit_card','month']).sum()
trans_sum.reset_index(inplace=True)

In [175]:
credit_new = pd.merge(credit, trans_sum, how='left', left_on='credit_card', right_on='credit_card')
credit_new['above_limit']=credit_new.credit_card_limit<credit_new.transaction_dollar_amount
credit_new.head()

,credit_card,city,state,zipcode,credit_card_limit,month,transaction_dollar_amount,above_limit
0,1280981422329509,Dallas,PA,18612,6000,7,950.65,False
1,1280981422329509,Dallas,PA,18612,6000,8,6171.35,True
2,1280981422329509,Dallas,PA,18612,6000,9,4681.90,False
3,1280981422329509,Dallas,PA,18612,6000,10,4963.99,False
4,9737219864179988,Houston,PA,15342,16000,7,283.62,False


In [176]:
limit_check=credit_new[['above_limit','credit_card']].groupby('credit_card').sum()
limit_check.reset_index(inplace=True)
limit_check.columns=['credit_card','times']
limit_check.head()

,credit_card,times
0,1003715054175576,0.0
1,1013870087888817,0.0
2,1023820165155391,0.0
3,1073931538936472,0.0
4,1077622576192810,0.0


In [177]:
id_list=limit_check[limit_check.times==0].credit_card
len(id_list)

862

## Question 2

### On the other hand, she wants you to implement an algorithm that as soon as a user goes above her monthly limit, it triggers an alert so that the user can be notiﬁed about that.We assume here that at the beginning of the new month, user total money spent gets reset to zero (i.e. she pays the card fully at the end of each month). Build a function that for each day, returns a list of users who went above their credit card monthly limit on that day.

In [130]:
fraud_list=limit_check[limit_check.times!=0]
fraud_list.head()

,credit_card,times
8,1106824181265726,3.0
20,1175962294549313,2.0
21,1190980117697422,1.0
28,1246716439259317,2.0
33,1280981422329509,1.0


In [178]:
def alert(data, month, day):
    """ function to return the list of credit cards that exceed the limit """
    data = data[(data['month'] == month) & (data['day'] <= day)]
    limit = data.groupby('credit_card')['credit_card_limit'].min().reset_index()
    transaction = data.groupby('credit_card')['transaction_dollar_amount'].sum().reset_index()
    
    merged = pd.merge(left=transaction, right=limit, on='credit_card', how='left')
    result = merged[merged['transaction_dollar_amount'] > merged['credit_card_limit']]
    
    return result

In [371]:
data = pd.merge(left=trans, right=credit, on='credit_card', how='left')
alert(data, month=9, day=20)

,credit_card,transaction_dollar_amount,credit_card_limit
144,2245942585429940,10415.62,10000
168,2505223645294729,4634.61,4000
240,3281814060807145,4792.53,4000
253,3369600965634913,2102.49,2000
337,4052848131106690,10335.37,9000
528,5723635641134781,2484.34,2000
578,6174559182308122,6360.36,6000
598,6292410823269309,2309.34,2000
639,6766253113444560,2578.53,2000
661,6984795534098127,12168.71,10000


## Question 3

### Finally, your boss is very concerned about frauds cause they are a huge cost for credit card companies. She wants you to implement an unsupervised algorithm that returns all transactions that seem unusual and are worth being investigated further.

There are two factors which impact a transaction is fraud or not:

1. if user spend the money far from his home, although it is possible due to traveling, but it's still very suspicious.

2. if the transaction violates that user's consumption habit. For example, if a user spend less then 200 each transaction most of the time, then a transaction more than 1000 will be highly suspicious.

Firstly, I start with the place where the money spent and the distance between home and money-spent place.

In [372]:
data.head()

,credit_card,transaction_dollar_amount,Long,Lat,month,day,weekday,hour,city,state,zipcode,credit_card_limit
0,1003715054175576,43.78,-80.174132,40.267370,9,11,4,0,Houston,PA,15342,20000
1,1003715054175576,103.15,-80.194240,40.180114,10,24,5,22,Houston,PA,15342,20000
2,1003715054175576,48.55,-80.211033,40.313004,10,26,0,18,Houston,PA,15342,20000
3,1003715054175576,136.18,-80.174138,40.290895,10,22,3,19,Houston,PA,15342,20000
4,1003715054175576,71.82,-80.238720,40.166719,10,26,0,20,Houston,PA,15342,20000


In [373]:
# calculate the median for Longitude and Latitude
address = data.groupby('credit_card')['Long', 'Lat'].median().reset_index()
address = address.rename(columns={'Long': 'Long_median', 'Lat': 'Lat_median'})
data = pd.merge(left=data, right=address, on='credit_card', how='left')

In [376]:
# calculate the distance
data['distance'] = np.sqrt((data['Long'] - data['Long_median'])**2 + (data['Lat'] - data['Lat_median'])**2)
data.head()

,credit_card,transaction_dollar_amount,Long,Lat,month,day,weekday,hour,city,state,zipcode,credit_card_limit,Long_median,Lat_median,distance
0,1003715054175576,43.78,-80.174132,40.267370,9,11,4,0,Houston,PA,15342,20000,-80.210281,40.242914,0.043645
1,1003715054175576,103.15,-80.194240,40.180114,10,24,5,22,Houston,PA,15342,20000,-80.210281,40.242914,0.064817
2,1003715054175576,48.55,-80.211033,40.313004,10,26,0,18,Houston,PA,15342,20000,-80.210281,40.242914,0.070094
3,1003715054175576,136.18,-80.174138,40.290895,10,22,3,19,Houston,PA,15342,20000,-80.210281,40.242914,0.060071
4,1003715054175576,71.82,-80.238720,40.166719,10,26,0,20,Houston,PA,15342,20000,-80.210281,40.242914,0.081329


In [377]:
data.distance.describe()

count    294588.000000
mean          1.993783
std          17.336845
min           0.000000
25%           0.044965
50%           0.063606
75%           0.078059
max         338.132513
Name: distance, dtype: float64

In [378]:
# drop useless features
labels = ['Long', 'Lat', 'city', 'state', 'zipcode', 'Long_median', 'Lat_median']
data = data.drop(labels=labels, axis=1)
data.head()

,credit_card,transaction_dollar_amount,month,day,weekday,hour,credit_card_limit,distance
0,1003715054175576,43.78,9,11,4,0,20000,0.043645
1,1003715054175576,103.15,10,24,5,22,20000,0.064817
2,1003715054175576,48.55,10,26,0,18,20000,0.070094
3,1003715054175576,136.18,10,22,3,19,20000,0.060071
4,1003715054175576,71.82,10,26,0,20,20000,0.081329


Then, we pay attention to the previous purchase habbit. We find the one quarter, one half, three quarters and the whole percentage of the purchasing value, then we compare the difference of these values. If the value is much larger than it usually be, we regard it as suspicious one.

In [379]:
def statistics_by_card(s):
    ps = [25,50,75,100]
    d = np.percentile(s,ps)
    return pd.Series(d,index=['{}%'.format(p) for p in ps])

tran_statistics = data.groupby('credit_card')['transaction_dollar_amount'].apply(statistics_by_card).unstack()

In [380]:
tran_statistics.head()

,25%,50%,75%,100%
credit_card,,,,
1003715054175576,59.01,93.73,124.63,995.35
1013870087888817,58.09,87.67,128.02,972.63
1023820165155391,55.43,85.78,122.83,977.59
1073931538936472,108.08,147.19,194.83,856.93
1077622576192810,122.93,148.18,160.11,177.55


In [381]:
data = pd.merge(data,tran_statistics,how='left',left_on='credit_card',right_index=True)

In [383]:
# drop useless features
labels = ['transaction_dollar_amount', 'credit_card_limit']
data = data.drop(labels=labels, axis=1)
data.head()

,credit_card,month,day,weekday,hour,distance,25%,50%,75%,100%
0,1003715054175576,9,11,4,0,0.043645,59.01,93.73,124.63,995.35
1,1003715054175576,10,24,5,22,0.064817,59.01,93.73,124.63,995.35
2,1003715054175576,10,26,0,18,0.070094,59.01,93.73,124.63,995.35
3,1003715054175576,10,22,3,19,0.060071,59.01,93.73,124.63,995.35
4,1003715054175576,10,26,0,20,0.081329,59.01,93.73,124.63,995.35


After data manipulating, we are able to implement the unsupervised algorithm that returns all transactions that seem unusual and are worth being investigated further. Since it's easy to find that if the max distance or the max transaction amount is far beyong the the maximum range or is several times of 75% value, it is more likely to be a fraud. However, it seems to be hard to cluster the suspucious behaviours and the normal ones, so I try to standardize the data and use DBSCAN to set every credit card as one group and fing the outliers.

In [246]:
address=data[['credit_card','transaction_dollar_amount','Long','Lat']]

In [302]:
address.head()

,credit_card,transaction_dollar_amount,Long,Lat
0,1003715054175576,43.78,-80.174132,40.267370
1,1003715054175576,103.15,-80.194240,40.180114
2,1003715054175576,48.55,-80.211033,40.313004
3,1003715054175576,136.18,-80.174138,40.290895
4,1003715054175576,71.82,-80.238720,40.166719


In [350]:
def outlier(data):
    place = np.vstack([data.Long.as_matrix(), data.Lat.as_matrix(), 
                       data.transaction_dollar_amount.as_matrix()]).transpose()
    index = data.index
    scaler = StandardScaler()
    data = scaler.fit_transform(place)
    cls= DBSCAN(eps=0.5)
    pred = cls.fit_predict(data)
    sus_index = index[pred==-1]
    return sus_index

In [313]:
credit_card_list = address.credit_card.unique()

In [365]:
suspucious_list = pd.Index([])
for card_number in address.credit_card.unique():
    card_data = address[address['credit_card'] == card_number]
    suspucious_list = suspucious_list.append(outlier(card_data))

In [367]:
suspucious_list

Int64Index([     8,     14,     84,    113,    128,    141,    145,    152,
               226,    246,
            ...
            294573, 294574, 294575, 294576, 294578, 294582, 294584, 294585,
            294586, 294587],
           dtype='int64', length=9360)

From the list above, each index represents one suspucious transaction.